In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/people/people.csv")

In [ ]:
df_sampled = df.iloc[1000:]

In [ ]:
import os
import requests
import re
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import time

def search_and_download_images(query, timeout=30):
    folder_name = query.replace(" ", "_")
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    url = f"https://www.google.com/search?hl=en&tbm=isch&q={query}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    
    try:
        response = requests.get(url, headers=headers, timeout=timeout)
        soup = BeautifulSoup(response.text, 'html.parser')

    except requests.exceptions.Timeout:
        return
    except Exception as e:
        return

    all_images = re.findall(r'https?://[^\s]*\.jpg', str(soup))

    start_time = time.time()
    images = []
    for img_url in all_images:
        if img_url.endswith('.jpg'):
            url_start = img_url.rfind("https://")
            url_end = img_url.find(".jpg") + 4
            img_url_clean = img_url[url_start:url_end]
            images.append(img_url_clean)
        elapsed_time = time.time() - start_time
        if elapsed_time >= timeout:
            break


    for i, img_url in tqdm(enumerate(images, start=1)):
        try:
            img_data = requests.get(img_url, timeout=3).content
            img = Image.open(BytesIO(img_data))
            img.save(os.path.join(folder_name, f"{i}.jpg"))
        except Exception as e:
            continue
        elapsed_time = time.time() - start_time
        if elapsed_time >= timeout:
            break

for name in df_sampled['name']:
    query = name.replace(" ", "+")
    search_and_download_images(query)